# E-Commerce Review Analyzer - Phase 1: Core AI Logic

**Project Goal:** Develop an AI system for automated e-commerce review analysis

**Core AI Functions:**
- Sentiment Analysis (Classification using DistilBERT)
- Abstractive Summarization (Generative AI using T5)

---

## Step 1.1: Environment Setup
Install required libraries for transformers, torch, and data handling

In [ ]:
# Install required packages
!pip install transformers torch pandas datasets matplotlib seaborn scikit-learn -q

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")

## Step 1.2: Data Ingestion
Load open-source Amazon Reviews dataset

In [ ]:
# Load Amazon Reviews dataset (using a subset for efficiency)
# Using the Amazon US Reviews dataset from Hugging Face
print("Loading dataset... This may take a moment.")

# Load a smaller subset: Amazon Polarity dataset (positive/negative reviews)
dataset = load_dataset("amazon_polarity", split="test[:5000]")  # Load 5000 reviews

# Convert to pandas DataFrame
df = pd.DataFrame(dataset)

# Rename columns for clarity
df.columns = ['label', 'title', 'content']

# Combine title and content for full review text
df['review_text'] = df['title'] + " " + df['content']

# Map labels: 0=Negative, 1=Positive (original dataset format)
df['original_sentiment'] = df['label'].map({0: 'negative', 1: 'positive'})

print(f"✓ Loaded {len(df)} reviews successfully!")
print(f"\nDataset shape: {df.shape}")
print(f"\nColumn names: {df.columns.tolist()}")
df.head(3)

In [ ]:
# Display basic statistics
print("Dataset Overview:")
print(f"Total Reviews: {len(df)}")
print(f"\nSentiment Distribution:")
print(df['original_sentiment'].value_counts())
print(f"\nAverage review length: {df['review_text'].str.len().mean():.0f} characters")

## Step 1.3: Sentiment Analysis LLM (Classification)
Implement sentiment classification using pre-trained DistilBERT model

In [ ]:
# Load pre-trained sentiment analysis model
print("Loading sentiment analysis model (DistilBERT)...")

# Using distilbert-base-uncased-finetuned-sst-2-english for sentiment analysis
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    truncation=True,
    max_length=512
)

print("✓ Sentiment model loaded successfully!")

In [ ]:
def classify_sentiment(text):
    """
    Classify sentiment of a given text using DistilBERT.
    
    Args:
        text (str): Review text to analyze
    
    Returns:
        dict: Contains 'label' (POSITIVE/NEGATIVE) and 'score' (confidence)
    """
    try:
        # Truncate text if too long
        text = text[:512] if len(text) > 512 else text
        
        # Get prediction
        result = sentiment_analyzer(text)[0]
        
        return {
            'label': result['label'],
            'score': round(result['score'], 4)
        }
    except Exception as e:
        return {
            'label': 'NEUTRAL',
            'score': 0.0,
            'error': str(e)
        }

# Test the function
test_text = "This product is amazing! I love it so much."
print("Test sentiment classification:")
print(f"Text: {test_text}")
print(f"Result: {classify_sentiment(test_text)}")

## Step 1.4: Summarization LLM (Generative AI)
Implement abstractive summarization using pre-trained T5 model

In [ ]:
# Load pre-trained summarization model
print("Loading summarization model (T5)...")

# Using T5-small for efficient summarization
summarizer = pipeline(
    "summarization",
    model="t5-small",
    truncation=True
)

print("✓ Summarization model loaded successfully!")

In [ ]:
def generate_summary(texts, max_length=150, min_length=40):
    """
    Generate abstractive summary from a list of texts using T5.
    
    Args:
        texts (list or str): Single text or list of texts to summarize
        max_length (int): Maximum length of summary
        min_length (int): Minimum length of summary
    
    Returns:
        str: Generated summary
    """
    try:
        # Handle single text or list
        if isinstance(texts, str):
            combined_text = texts
        else:
            combined_text = " ".join(texts)
        
        # Truncate if too long (T5 has token limits)
        if len(combined_text) > 1000:
            combined_text = combined_text[:1000]
        
        # Generate summary
        summary = summarizer(
            combined_text,
            max_length=max_length,
            min_length=min_length,
            do_sample=False
        )
        
        return summary[0]['summary_text']
    
    except Exception as e:
        return f"Error generating summary: {str(e)}"

# Test the function
test_reviews = [
    "This product exceeded my expectations. The quality is outstanding and shipping was fast.",
    "Great value for money. Would definitely recommend to friends and family.",
    "The customer service was excellent. They helped resolve my issue quickly."
]
print("Test summarization:")
print(f"Number of reviews: {len(test_reviews)}")
print(f"\nGenerated Summary:\n{generate_summary(test_reviews)}")

## Step 1.5: Functional Proof - Test on 10 Random Reviews
Validate both LLM functions on sample data

In [ ]:
# Select 10 random reviews for testing
test_sample = df.sample(n=10, random_state=42).reset_index(drop=True)

print("=" * 80)
print("FUNCTIONAL PROOF: Testing Both LLM Functions on 10 Random Reviews")
print("=" * 80)

# Test sentiment classification on each review
print("\n1. SENTIMENT ANALYSIS TEST:")
print("-" * 80)

sentiment_results = []
for idx, row in test_sample.iterrows():
    review_text = row['review_text'][:200]  # Show first 200 chars
    sentiment = classify_sentiment(row['review_text'])
    sentiment_results.append(sentiment)
    
    print(f"\nReview {idx + 1}:")
    print(f"Text: {review_text}...")
    print(f"Original Label: {row['original_sentiment']}")
    print(f"Predicted Sentiment: {sentiment['label']} (Confidence: {sentiment['score']})")

# Add results to dataframe
test_sample['predicted_sentiment'] = [r['label'] for r in sentiment_results]
test_sample['confidence_score'] = [r['score'] for r in sentiment_results]

In [ ]:
# Test summarization on positive and negative reviews separately
print("\n" + "=" * 80)
print("2. SUMMARIZATION TEST:")
print("-" * 80)

# Get positive reviews from test sample
positive_reviews = test_sample[test_sample['original_sentiment'] == 'positive']['review_text'].tolist()
negative_reviews = test_sample[test_sample['original_sentiment'] == 'negative']['review_text'].tolist()

print(f"\n📊 Positive Reviews ({len(positive_reviews)} reviews):")
if positive_reviews:
    positive_summary = generate_summary(positive_reviews[:5])  # Summarize up to 5 reviews
    print(f"Summary: {positive_summary}")
else:
    print("No positive reviews in sample.")

print(f"\n📊 Negative Reviews ({len(negative_reviews)} reviews):")
if negative_reviews:
    negative_summary = generate_summary(negative_reviews[:5])  # Summarize up to 5 reviews
    print(f"Summary: {negative_summary}")
else:
    print("No negative reviews in sample.")

In [ ]:
# Display validation summary
print("\n" + "=" * 80)
print("VALIDATION SUMMARY:")
print("=" * 80)

print("\n✓ Sentiment Classification Function: WORKING")
print(f"  - Successfully processed {len(test_sample)} reviews")
print(f"  - Output format: dict with 'label' and 'score' keys")
print(f"  - Average confidence: {test_sample['confidence_score'].mean():.2%}")

print("\n✓ Summarization Function: WORKING")
print(f"  - Successfully generated summaries for positive and negative reviews")
print(f"  - Output format: string (abstractive summary)")

print("\n🎉 PHASE 1 COMPLETE: Core AI Logic Validated!")

## Bonus: Visualize Results

In [ ]:
# Visualize sentiment distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Original sentiment distribution
test_sample['original_sentiment'].value_counts().plot(
    kind='bar', 
    ax=axes[0], 
    color=['#ff6b6b', '#51cf66'],
    alpha=0.7
)
axes[0].set_title('Original Sentiment Labels', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Sentiment')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=0)

# Predicted sentiment distribution
test_sample['predicted_sentiment'].value_counts().plot(
    kind='bar', 
    ax=axes[1], 
    color=['#ff6b6b', '#51cf66'],
    alpha=0.7
)
axes[1].set_title('Predicted Sentiment (DistilBERT)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Sentiment')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

# Display confidence score distribution
plt.figure(figsize=(10, 5))
plt.hist(test_sample['confidence_score'], bins=10, color='#4c6ef5', alpha=0.7, edgecolor='black')
plt.title('Model Confidence Score Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Confidence Score')
plt.ylabel('Frequency')
plt.axvline(test_sample['confidence_score'].mean(), color='red', linestyle='--', label=f'Mean: {test_sample["confidence_score"].mean():.3f}')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.show()

## Save Processed Data for Phase 2

In [ ]:
# Save a subset of processed data for use in Streamlit dashboard
# Process sentiment for 1000 reviews (more efficient for dashboard)
sample_for_dashboard = df.sample(n=1000, random_state=42).reset_index(drop=True)

print("Processing sentiment for 1000 reviews... This may take a few minutes.")

# Process in batches
sentiments = []
for idx, text in enumerate(sample_for_dashboard['review_text']):
    if idx % 100 == 0:
        print(f"Processed {idx}/{len(sample_for_dashboard)} reviews...")
    sentiments.append(classify_sentiment(text))

sample_for_dashboard['sentiment'] = [s['label'] for s in sentiments]
sample_for_dashboard['confidence'] = [s['score'] for s in sentiments]

# Save to CSV
sample_for_dashboard.to_csv('processed_reviews.csv', index=False)
print("\n✓ Saved processed_reviews.csv for Phase 2 dashboard!")